# Detect malfunctioning sensors in real time using Streaming Analytics and Python


In this notebook, you will create an application that receives weather data from simulated weather stations and then detects if any of those stations are malfunctioning.  This is done by comparing the temperature from each station with the average temperature from all the other stations in the same region. If a station's reading is considered to be an outlier, then it is flagged as potentially malfunctioning.

The data is visualized on a map, and malfunctioning stations are shown as red, as in the image below. Note that the readings are updated in real time.
<img src="https://raw.githubusercontent.com/IBMStreams/samples/master/IoT/WeatherStationApp/img/dsx-weather-app.gif"/>

## Notebook has moved
This notebook is now [hosted on IBM Watson Studio](https://dataplatform.ibm.com/exchange/public/entry/view/ec0aa15c6ab928b9b43ac0109d4395f1).

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the Apache License.